# 戦略最適化 サンプルノートブック

このノートブックでは、`StrategyOptimizer`を使った投資戦略パラメータの自動最適化を学びます。

## 使用するクラス

| クラス | 主な用途 |
|--------|----------|
| **StrategyOptimizer** | 戦略パラメータの最適化エンジン |
| **OptimizationResults** | 最適化結果の分析・可視化・保存 |
| **TrialResult** | 個別試行の結果データ |

## 前提条件
- J-Quants APIでデータ収集済み（`data/jquants.db`が存在）
- 分析スクリプト実行済み（`data/analysis_results.db`が存在）

In [1]:
# 基本インポート
import pandas as pd

# Bokeh警告を抑制（backtesting.pyが内部で使用）
import backtesting

backtesting.set_bokeh_output(notebook=False)

# 最適化関連クラスをインポート
from technical_tools import (
    StrategyOptimizer,
    OptimizationResults,
    TrialResult,
    OptimizationTimeoutError,
    InvalidSearchSpaceError,
    NoValidParametersError,
)

print("インポート完了")

/Users/tak/Markets/Stocks/stock-analysis/.venv/lib/python3.10/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

インポート完了


---
## 2. StrategyOptimizer: 基本

`StrategyOptimizer`は内部で`Backtester`を使用し、複数のパラメータ組み合わせを効率的に評価して最適な戦略を発見します。

### 初期化パラメータ
| パラメータ | 説明 | デフォルト |
|------------|------|------------|
| `cash` | 初期資金 | 1,000,000円 |
| `commission` | 手数料率 | 0（無料） |

In [2]:
# StrategyOptimizerの初期化
optimizer = StrategyOptimizer(cash=1_000_000, commission=0.001)  # 0.1%の手数料
print(f"Optimizer初期化: {optimizer}")

Optimizer初期化: StrategyOptimizer(search_spaces=[], constraints=0)


---
## 3. 探索空間の定義

`add_search_space()`メソッドで最適化するパラメータと候補値を定義します。

### 対応パラメータ一覧
| パラメータ名 | 説明 | 用途 |
|--------------|------|------|
| `ma_short` | 短期移動平均期間 | ゴールデンクロス |
| `ma_long` | 長期移動平均期間 | ゴールデンクロス |
| `rsi_threshold` | RSI閾値 | RSI売られすぎシグナル |
| `macd_fast` | MACD短期EMA期間 | MACDクロス |
| `macd_slow` | MACD長期EMA期間 | MACDクロス |
| `macd_signal` | MACDシグナル期間 | MACDクロス |
| `stop_loss` | 損切り閾値（負の値） | 退出ルール |
| `take_profit` | 利確閾値（正の値） | 退出ルール |

In [3]:
# 探索空間の定義
optimizer = StrategyOptimizer(cash=1_000_000)

# 短期・長期MAの候補
optimizer.add_search_space("ma_short", [5, 10, 20, 25])
optimizer.add_search_space("ma_long", [50, 75, 100, 200])

# 損切りラインの候補
optimizer.add_search_space("stop_loss", [-0.05, -0.08, -0.10, -0.12])

print(f"探索空間: {optimizer}")
print(f"全組み合わせ数: {4 * 4 * 4} = 64通り")

探索空間: StrategyOptimizer(search_spaces=['ma_short', 'ma_long', 'stop_loss'], constraints=0)
全組み合わせ数: 64 = 64通り


In [4]:
# メソッドチェーンで定義（同等の書き方）
optimizer2 = (
    StrategyOptimizer()
    .add_search_space("ma_short", [5, 10, 20])
    .add_search_space("ma_long", [50, 75, 100])
    .add_search_space("stop_loss", [-0.08, -0.10])
)
print(f"チェーン定義: {optimizer2}")

チェーン定義: StrategyOptimizer(search_spaces=['ma_short', 'ma_long', 'stop_loss'], constraints=0)


---
## 4. 制約条件の追加

`add_constraint()`で無効なパラメータ組み合わせを除外できます。
制約関数はパラメータ辞書を受け取り、有効なら`True`を返します。

In [5]:
# 制約条件の追加: 短期 < 長期
optimizer.add_constraint(lambda p: p["ma_short"] < p["ma_long"])

# 複数の制約を追加可能
optimizer.add_constraint(lambda p: p["ma_long"] >= 2 * p["ma_short"])  # 長期は短期の2倍以上

print(f"制約追加後: {optimizer}")

制約追加後: StrategyOptimizer(search_spaces=['ma_short', 'ma_long', 'stop_loss'], constraints=2)


In [6]:
# 複雑な制約の例
optimizer3 = StrategyOptimizer()
optimizer3.add_search_space("ma_short", [5, 10, 20])
optimizer3.add_search_space("ma_long", [50, 75, 100])
optimizer3.add_search_space("stop_loss", [-0.05, -0.10, -0.15])
optimizer3.add_search_space("take_profit", [0.10, 0.15, 0.20])

# 複合制約: リスク・リワード比が1.5以上
optimizer3.add_constraint(
    lambda p: p["take_profit"] >= abs(p["stop_loss"]) * 1.5
)

print("リスク・リワード制約を追加")

リスク・リワード制約を追加


---
## 5. グリッドサーチ最適化

`method="grid"`（デフォルト）で全パラメータ組み合わせを探索します。
探索空間が小さい場合に最適です。

In [7]:
# グリッドサーチの実行
optimizer = StrategyOptimizer(cash=1_000_000)
optimizer.add_search_space("ma_short", [5, 10, 20])
optimizer.add_search_space("ma_long", [50, 75])
optimizer.add_search_space("stop_loss", [-0.08, -0.10])
optimizer.add_constraint(lambda p: p["ma_short"] < p["ma_long"])

print("グリッドサーチ実行中...")

try:
    results = optimizer.run(
        symbols=["7203"],  # トヨタ
        start="2024-01-01",
        end="2024-12-31",
        method="grid",     # 全組み合わせ探索
        metric="sharpe_ratio",
    )
    print(f"最適化完了: {results}")
except Exception as e:
    print(f"エラー: {e}")
    results = None

グリッドサーチ実行中...


Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

...

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

最適化完了: OptimizationResults(trials=12, metric=sharpe_ratio)


---
## 6. ランダムサーチ最適化

`method="random"`で指定回数のランダムサンプリングを行います。
探索空間が大きい場合に効率的です。

In [8]:
# ランダムサーチの実行
optimizer_rand = StrategyOptimizer(cash=1_000_000)
optimizer_rand.add_search_space("ma_short", [5, 10, 15, 20, 25])
optimizer_rand.add_search_space("ma_long", [50, 60, 75, 100, 150, 200])
optimizer_rand.add_search_space("stop_loss", [-0.05, -0.08, -0.10, -0.12, -0.15])
optimizer_rand.add_constraint(lambda p: p["ma_short"] < p["ma_long"])

print(f"全組み合わせ: 5 x 6 x 5 = 150通り")
print("ランダムに20回サンプリング...")

try:
    results_rand = optimizer_rand.run(
        symbols=["7203"],
        start="2024-01-01",
        end="2024-12-31",
        method="random",   # ランダムサンプリング
        n_trials=20,       # 試行回数
        metric="sharpe_ratio",
    )
    print(f"ランダムサーチ完了: {results_rand}")
except Exception as e:
    print(f"エラー: {e}")
    results_rand = None

全組み合わせ: 5 x 6 x 5 = 150通り
ランダムに20回サンプリング...


Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

...

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

ランダムサーチ完了: OptimizationResults(trials=20, metric=sharpe_ratio)


---
## 7. 最適化メトリクス

最適化の目的関数として使用できるメトリクス一覧です。

| メトリクス | 説明 | 最適化方向 |
|-----------|------|------------|
| `sharpe_ratio` | シャープレシオ | 最大化 |
| `total_return` | トータルリターン | 最大化 |
| `win_rate` | 勝率 | 最大化 |
| `profit_factor` | プロフィットファクター | 最大化 |
| `max_drawdown` | 最大ドローダウン | 最小化 |

In [9]:
# 異なるメトリクスでの最適化
optimizer_wr = StrategyOptimizer()
optimizer_wr.add_search_space("ma_short", [5, 10, 20])
optimizer_wr.add_search_space("ma_long", [50, 75])
optimizer_wr.add_constraint(lambda p: p["ma_short"] < p["ma_long"])

print("勝率(win_rate)を最適化...")

try:
    results_wr = optimizer_wr.run(
        symbols=["7203"],
        start="2024-01-01",
        end="2024-12-31",
        metric="win_rate",  # 勝率を最適化
    )
    print(f"勝率最適化完了: {results_wr}")
except Exception as e:
    print(f"エラー: {e}")

勝率(win_rate)を最適化...


Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

勝率最適化完了: OptimizationResults(trials=6, metric=win_rate)


---
## 8. 複合メトリクス（重み付け）

複数のメトリクスを重み付けして最適化できます。
結果には`composite_score`が追加されます。

In [10]:
# 複合メトリクスでの最適化
optimizer_comp = StrategyOptimizer()
optimizer_comp.add_search_space("ma_short", [5, 10, 20])
optimizer_comp.add_search_space("ma_long", [50, 75])
optimizer_comp.add_constraint(lambda p: p["ma_short"] < p["ma_long"])

print("複合メトリクス（シャープ50% + 勝率30% + ドローダウン20%）で最適化...")

try:
    results_comp = optimizer_comp.run(
        symbols=["7203"],
        start="2024-01-01",
        end="2024-12-31",
        metric={
            "sharpe_ratio": 0.5,   # 50%
            "win_rate": 0.3,       # 30%
            "max_drawdown": 0.2,   # 20%（自動的に反転）
        },
    )
    print(f"複合最適化完了: {results_comp}")
except Exception as e:
    print(f"エラー: {e}")

複合メトリクス（シャープ50% + 勝率30% + ドローダウン20%）で最適化...


Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

複合最適化完了: OptimizationResults(trials=6, metric={'sharpe_ratio': 0.5, 'win_rate': 0.3, 'max_drawdown': 0.2})


In [11]:
# 複合メトリクスの結果にはcomposite_scoreが含まれる
if results_comp:
    top_df = results_comp.top(5)
    print("=== 複合スコアトップ5 ===")
    display(top_df)

=== 複合スコアトップ5 ===


,ma_short,ma_long,total_return,sharpe_ratio,max_drawdown,win_rate,profit_factor,composite_score
0,20,50,0.0024,1.65,0.0761,1.0,99.99,1.30978
1,5,75,0.0021,1.62,0.0762,1.0,99.99,1.29476
2,20,75,0.0017,1.52,0.0654,1.0,99.99,1.24692
3,10,50,0.0018,1.39,0.0761,1.0,99.99,1.17978
4,5,50,0.0017,1.36,0.0762,1.0,99.99,1.16476


---
## 9. OptimizationResults: 結果分析

`OptimizationResults`クラスは最適化結果の分析・可視化機能を提供します。

### 主なメソッド
| メソッド | 説明 |
|----------|------|
| `best()` | 最良の結果を取得 |
| `top(n)` | 上位N件をDataFrameで取得 |
| `plot_heatmap()` | パラメータ空間のヒートマップ |
| `save()` | 結果をJSON/CSVに保存 |
| `load()` | 保存した結果を読み込み |

In [12]:
# best(): 最良の結果
if results:
    best = results.best()
    if best:
        print("=== 最良パラメータ ===")
        print(f"パラメータ: {best.params}")
        print(f"メトリクス:")
        for key, value in best.metrics.items():
            if isinstance(value, float):
                print(f"  {key}: {value:.4f}")
            else:
                print(f"  {key}: {value}")

=== 最良パラメータ ===
パラメータ: {'ma_short': 20, 'ma_long': 50, 'stop_loss': -0.08}
メトリクス:
  total_return: 0.0024
  sharpe_ratio: 1.6500
  max_drawdown: 0.0761
  win_rate: 1.0000
  profit_factor: 99.9900


In [13]:
# top(): 上位N件をDataFrameで取得
if results:
    top_10 = results.top(10)
    print("=== 上位10件 ===")
    display(top_10)

=== 上位10件 ===


,ma_short,ma_long,stop_loss,total_return,sharpe_ratio,max_drawdown,win_rate,profit_factor
0,20,50,-0.08,0.0024,1.65,0.0761,1.0,99.99
1,20,50,-0.10,0.0024,1.65,0.0761,1.0,99.99
2,5,75,-0.08,0.0021,1.62,0.0762,1.0,99.99
3,5,75,-0.10,0.0021,1.62,0.0762,1.0,99.99
4,20,75,-0.10,0.0017,1.52,0.0654,1.0,99.99
5,20,75,-0.08,0.0017,1.52,0.0654,1.0,99.99
6,10,50,-0.10,0.0018,1.39,0.0761,1.0,99.99
7,10,50,-0.08,0.0018,1.39,0.0761,1.0,99.99
8,5,50,-0.08,0.0017,1.36,0.0762,1.0,99.99
9,5,50,-0.10,0.0017,1.36,0.0762,1.0,99.99


In [14]:
# TrialResultの構造
if results:
    best = results.best()
    if best:
        print("=== TrialResultの構造 ===")
        print(f"params: {type(best.params)} - パラメータ辞書")
        print(f"metrics: {type(best.metrics)} - 評価指標辞書")
        print(f"oos_metrics: {type(best.oos_metrics)} - OOS評価（walk-forward時）")
        print(f"backtest_results: {type(best.backtest_results)} - 詳細結果（通常None）")

=== TrialResultの構造 ===
params: <class 'dict'> - パラメータ辞書
metrics: <class 'dict'> - 評価指標辞書
oos_metrics: <class 'NoneType'> - OOS評価（walk-forward時）
backtest_results: <class 'NoneType'> - 詳細結果（通常None）


---
## 10. ヒートマップ可視化

`plot_heatmap()`でパラメータ空間と評価指標の関係を可視化できます。
`max_drawdown`の場合は自動的にカラースケールが反転します。

In [15]:
# ヒートマップの生成
if results:
    fig = results.plot_heatmap(
        x_param="ma_short",
        y_param="ma_long",
        metric="sharpe_ratio",
    )
    fig.update_layout(height=400)
    fig.show()

In [16]:
# max_drawdownのヒートマップ（カラースケール反転）
if results:
    fig_dd = results.plot_heatmap(
        x_param="ma_short",
        y_param="stop_loss",
        metric="max_drawdown",
    )
    fig_dd.update_layout(
        height=400,
        title="最大ドローダウン（低いほど良い）",
    )
    fig_dd.show()

---
## 11. ウォークフォワード検証

過学習を防ぐため、`validation="walk_forward"`でアウトオブサンプル評価を行えます。

### パラメータ
| パラメータ | 説明 | デフォルト |
|------------|------|------------|
| `train_ratio` | 学習期間の割合 | 0.7（70%） |
| `n_splits` | 分割数 | 5 |

In [17]:
# ウォークフォワード検証
optimizer_wf = StrategyOptimizer()
optimizer_wf.add_search_space("ma_short", [5, 10, 20])
optimizer_wf.add_search_space("ma_long", [50, 75])
optimizer_wf.add_constraint(lambda p: p["ma_short"] < p["ma_long"])

print("ウォークフォワード検証付き最適化...")

try:
    results_wf = optimizer_wf.run(
        symbols=["7203"],
        start="2023-01-01",
        end="2024-12-31",  # 2年間のデータ
        validation="walk_forward",
        train_ratio=0.7,  # 70%を学習、30%をテスト
        n_splits=5,       # 5分割
    )
    print(f"ウォークフォワード完了: {results_wf}")
except Exception as e:
    print(f"エラー: {e}")
    results_wf = None

ウォークフォワード検証付き最適化...


Backtest.run:   0%|          | 0/490 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/490 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/29 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/490 [00:00<?, ?bar/s]

...

Backtest.run:   0%|          | 0/30 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/30 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/30 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/30 [00:00<?, ?bar/s]

ウォークフォワード完了: OptimizationResults(trials=6, metric=sharpe_ratio)


In [18]:
# OOSメトリクスの確認
if results_wf:
    best_wf = results_wf.best()
    if best_wf:
        print("=== ウォークフォワード結果 ===")
        print(f"パラメータ: {best_wf.params}")
        print("\nイン・サンプル評価:")
        for k, v in best_wf.metrics.items():
            print(f"  {k}: {v:.4f}")
        
        if best_wf.oos_metrics:
            print("\nアウト・オブ・サンプル評価:")
            for k, v in best_wf.oos_metrics.items():
                print(f"  {k}: {v:.4f}")

=== ウォークフォワード結果 ===
パラメータ: {'ma_short': 5, 'ma_long': 50}

イン・サンプル評価:
  total_return: 0.0070
  sharpe_ratio: 1.0200
  max_drawdown: 0.4232
  win_rate: 1.0000
  profit_factor: 99.9900

アウト・オブ・サンプル評価:
  total_return: 0.0000
  sharpe_ratio: 0.0000
  win_rate: 0.0000


---
## 12. 並列実行

`n_jobs`パラメータで並列実行のワーカー数を制御できます。

| 値 | 動作 |
|----|------|
| `-1` | 全CPUコア使用（デフォルト） |
| `1` | シングルスレッド |
| `N` | Nワーカーで並列実行 |

In [19]:
# 並列実行の制御
import os

print(f"利用可能CPUコア数: {os.cpu_count()}")

optimizer_par = StrategyOptimizer()
optimizer_par.add_search_space("ma_short", [5, 10, 20])
optimizer_par.add_search_space("ma_long", [50, 75])
optimizer_par.add_constraint(lambda p: p["ma_short"] < p["ma_long"])

print("\n4ワーカーで並列実行...")

try:
    results_par = optimizer_par.run(
        symbols=["7203"],
        start="2024-01-01",
        end="2024-12-31",
        n_jobs=4,  # 4並列
    )
    print(f"並列実行完了: {results_par}")
except Exception as e:
    print(f"エラー: {e}")

利用可能CPUコア数: 16

4ワーカーで並列実行...


Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

並列実行完了: OptimizationResults(trials=6, metric=sharpe_ratio)


---
## 13. タイムアウト処理

`timeout`パラメータで最適化の最大実行時間（秒）を設定できます。
タイムアウト時は`OptimizationTimeoutError`が発生します。

In [20]:
# タイムアウト設定
optimizer_to = StrategyOptimizer()
optimizer_to.add_search_space("ma_short", [5, 10, 20, 25])
optimizer_to.add_search_space("ma_long", [50, 75, 100, 200])
optimizer_to.add_search_space("stop_loss", [-0.05, -0.08, -0.10, -0.12])
optimizer_to.add_constraint(lambda p: p["ma_short"] < p["ma_long"])

print("5秒のタイムアウトで実行...")

try:
    results_to = optimizer_to.run(
        symbols=["7203", "9984"],  # 複数銘柄で時間がかかる
        start="2024-01-01",
        end="2024-12-31",
        timeout=5.0,  # 5秒でタイムアウト
    )
    print(f"完了: {results_to}")
except OptimizationTimeoutError as e:
    print(f"タイムアウト: {e.completed}/{e.total}件完了")
    print(f"  経過時間: {e.timeout}秒")

5秒のタイムアウトで実行...


Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

...

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

完了: OptimizationResults(trials=64, metric=sharpe_ratio)


---
## 14. ストリーミング出力

`streaming_output`パラメータで試行結果を逐次JSONL形式で保存できます。
大量の試行時にメモリ効率が良くなります。

In [21]:
# ストリーミング出力
from pathlib import Path

output_dir = Path("output")
output_dir.mkdir(exist_ok=True)

optimizer_stream = StrategyOptimizer()
optimizer_stream.add_search_space("ma_short", [5, 10, 20])
optimizer_stream.add_search_space("ma_long", [50, 75])
optimizer_stream.add_constraint(lambda p: p["ma_short"] < p["ma_long"])

print("ストリーミング出力で実行...")

try:
    results_stream = optimizer_stream.run(
        symbols=["7203"],
        start="2024-01-01",
        end="2024-12-31",
        streaming_output="output/optimization_stream.jsonl",
    )
    print(f"完了: {results_stream}")
    print(f"ストリーミング出力: output/optimization_stream.jsonl")
except Exception as e:
    print(f"エラー: {e}")

ストリーミング出力で実行...


Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

完了: OptimizationResults(trials=6, metric=sharpe_ratio)
ストリーミング出力: output/optimization_stream.jsonl


In [22]:
# JONLファイルから結果を読み込み
jsonl_path = Path("output/optimization_stream.jsonl")

if jsonl_path.exists():
    loaded_stream = OptimizationResults.load_streaming(
        jsonl_path,
        metric="sharpe_ratio",
    )
    print(f"読み込み完了: {loaded_stream}")
    display(loaded_stream.top(5))
else:
    print("ストリーミング出力ファイルが存在しません")

読み込み完了: OptimizationResults(trials=6, metric=sharpe_ratio)


,ma_short,ma_long,total_return,sharpe_ratio,max_drawdown,win_rate,profit_factor
0,20,50,0.0024,1.65,0.0761,1.0,99.99
1,5,75,0.0021,1.62,0.0762,1.0,99.99
2,20,75,0.0017,1.52,0.0654,1.0,99.99
3,10,50,0.0018,1.39,0.0761,1.0,99.99
4,5,50,0.0017,1.36,0.0762,1.0,99.99


---
## 15. 結果の保存・読み込み

`save()`と`load()`で最適化結果を永続化できます。

| 形式 | 拡張子 | 特徴 |
|------|--------|------|
| JSON | `.json` | 完全な情報を保存（推奨） |
| CSV | `.csv` | 表形式、他ツールと連携 |

In [23]:
# 結果の保存
if results:
    # JSON形式で保存
    json_path = results.save("output/optimization_results.json")
    print(f"JSON保存: {json_path}")
    
    # CSV形式で保存
    csv_path = results.save("output/optimization_results.csv")
    print(f"CSV保存: {csv_path}")

JSON保存: output/optimization_results.json
CSV保存: output/optimization_results.csv


In [24]:
# 保存した結果の読み込み
json_path = Path("output/optimization_results.json")

if json_path.exists():
    loaded = OptimizationResults.load(json_path)
    print(f"読み込み完了: {loaded}")
    print(f"最良パラメータ: {loaded.best().params}")
else:
    print("保存ファイルが存在しません")

読み込み完了: OptimizationResults(trials=12, metric=sharpe_ratio)
最良パラメータ: {'ma_short': 20, 'ma_long': 50, 'stop_loss': -0.08}


---
## 16. RSI戦略の最適化

RSI閾値と損切り/利確ラインを同時に最適化する例です。

In [25]:
# RSI戦略の最適化
optimizer_rsi = StrategyOptimizer(cash=1_000_000)

# RSI閾値（売られすぎ）
optimizer_rsi.add_search_space("rsi_threshold", [20, 25, 30, 35])

# 損切り・利確
optimizer_rsi.add_search_space("stop_loss", [-0.05, -0.08, -0.10])
optimizer_rsi.add_search_space("take_profit", [0.10, 0.15, 0.20])

print(f"RSI戦略最適化: {4 * 3 * 3} = 36通り")

try:
    results_rsi = optimizer_rsi.run(
        symbols=["7203", "9984"],
        start="2024-01-01",
        end="2024-12-31",
        metric="profit_factor",
    )
    print(f"\nRSI最適化完了: {results_rsi}")
    
    best_rsi = results_rsi.best()
    if best_rsi:
        print(f"\n最良パラメータ:")
        print(f"  RSI閾値: {best_rsi.params['rsi_threshold']}")
        print(f"  損切り: {best_rsi.params['stop_loss']:.0%}")
        print(f"  利確: {best_rsi.params['take_profit']:.0%}")
        print(f"  プロフィットファクター: {best_rsi.metrics['profit_factor']:.2f}")
except Exception as e:
    print(f"エラー: {e}")

RSI戦略最適化: 36 = 36通り


Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

...

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]


RSI最適化完了: OptimizationResults(trials=36, metric=profit_factor)

最良パラメータ:
  RSI閾値: 20
  損切り: -8%
  利確: 15%
  プロフィットファクター: 99.99


---
## 17. MACD戦略の最適化

MACDのfast/slow/signalパラメータを最適化する例です。

In [26]:
# MACD戦略の最適化
optimizer_macd = StrategyOptimizer(cash=1_000_000)

# MACDパラメータ
optimizer_macd.add_search_space("macd_fast", [8, 12, 16])
optimizer_macd.add_search_space("macd_slow", [20, 26, 30])
optimizer_macd.add_search_space("macd_signal", [7, 9, 11])

# 制約: fast < slow
optimizer_macd.add_constraint(lambda p: p["macd_fast"] < p["macd_slow"])

print("MACD戦略最適化...")

try:
    results_macd = optimizer_macd.run(
        symbols=["7203"],
        start="2024-01-01",
        end="2024-12-31",
        metric="sharpe_ratio",
    )
    print(f"\nMACD最適化完了: {results_macd}")
    
    best_macd = results_macd.best()
    if best_macd:
        print(f"\n最良MACD設定:")
        print(f"  Fast: {best_macd.params['macd_fast']}")
        print(f"  Slow: {best_macd.params['macd_slow']}")
        print(f"  Signal: {best_macd.params['macd_signal']}")
        print(f"  シャープレシオ: {best_macd.metrics['sharpe_ratio']:.2f}")
except Exception as e:
    print(f"エラー: {e}")

MACD戦略最適化...


Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

...

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]


MACD最適化完了: OptimizationResults(trials=27, metric=sharpe_ratio)

最良MACD設定:
  Fast: 12
  Slow: 26
  Signal: 9
  シャープレシオ: 0.63


---
## 18. 実践ワークフロー

実際の最適化では、以下のステップが推奨されます：

1. **Step1**: 探索空間の定義
2. **Step2**: 最適化実行（ウォークフォワード付き）
3. **Step3**: 結果分析
4. **Step4**: ヒートマップ可視化
5. **Step5**: 結果保存

In [27]:
# Step1: 探索空間の定義
print("=== Step1: 探索空間の定義 ===")

workflow_opt = StrategyOptimizer(cash=1_000_000, commission=0.001)

# MAクロスパラメータ
workflow_opt.add_search_space("ma_short", [5, 10, 20, 25])
workflow_opt.add_search_space("ma_long", [50, 75, 100])

# 損切りライン
workflow_opt.add_search_space("stop_loss", [-0.08, -0.10, -0.12])

# 制約
workflow_opt.add_constraint(lambda p: p["ma_short"] < p["ma_long"])

print(f"探索空間: {workflow_opt}")
print(f"パラメータ組み合わせ: 4 x 3 x 3 = 36通り")

=== Step1: 探索空間の定義 ===
探索空間: StrategyOptimizer(search_spaces=['ma_short', 'ma_long', 'stop_loss'], constraints=1)
パラメータ組み合わせ: 4 x 3 x 3 = 36通り


In [28]:
# Step2: 最適化実行（ウォークフォワード付き）
print("=== Step2: 最適化実行 ===")

try:
    workflow_results = workflow_opt.run(
        symbols=["7203", "9984"],
        start="2023-01-01",
        end="2024-12-31",
        method="grid",
        metric={
            "sharpe_ratio": 0.5,
            "win_rate": 0.3,
            "max_drawdown": 0.2,
        },
        validation="walk_forward",
        train_ratio=0.7,
        n_splits=4,
        n_jobs=4,
    )
    print(f"最適化完了: {workflow_results}")
except Exception as e:
    print(f"エラー: {e}")
    workflow_results = None

=== Step2: 最適化実行 ===


Backtest.run:   0%|          | 0/490 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/490 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/490 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/490 [00:00<?, ?bar/s]

...

Backtest.run:   0%|          | 0/38 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/38 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/38 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/38 [00:00<?, ?bar/s]

最適化完了: OptimizationResults(trials=36, metric={'sharpe_ratio': 0.5, 'win_rate': 0.3, 'max_drawdown': 0.2})


In [29]:
# Step3: 結果分析
print("=== Step3: 結果分析 ===")

if workflow_results:
    best = workflow_results.best()
    
    if best:
        print("\n【最良パラメータ】")
        for k, v in best.params.items():
            print(f"  {k}: {v}")
        
        print("\n【イン・サンプル評価】")
        for k, v in best.metrics.items():
            print(f"  {k}: {v:.4f}")
        
        if best.oos_metrics:
            print("\n【アウト・オブ・サンプル評価】")
            for k, v in best.oos_metrics.items():
                print(f"  {k}: {v:.4f}")
    
    print("\n【上位5件】")
    display(workflow_results.top(5))

=== Step3: 結果分析 ===

【最良パラメータ】
  ma_short: 5
  ma_long: 50
  stop_loss: -0.12

【イン・サンプル評価】
  total_return: 0.0073
  sharpe_ratio: 1.0800
  max_drawdown: 0.4008
  win_rate: 1.0000
  profit_factor: 99.9900

【アウト・オブ・サンプル評価】
  total_return: 0.0000
  sharpe_ratio: 0.0000
  win_rate: 0.0000

【上位5件】


,ma_short,ma_long,stop_loss,total_return,sharpe_ratio,max_drawdown,win_rate,profit_factor,composite_score
0,5,50,-0.12,0.0073,1.08,0.4008,1.0,99.99,0.95984
1,5,50,-0.08,0.0073,1.08,0.4008,1.0,99.99,0.95984
2,5,50,-0.10,0.0073,1.08,0.4008,1.0,99.99,0.95984
3,25,50,-0.10,0.0074,1.08,0.4028,1.0,99.99,0.95944
4,25,50,-0.08,0.0074,1.08,0.4028,1.0,99.99,0.95944


In [30]:
# Step4: ヒートマップ可視化
print("=== Step4: ヒートマップ可視化 ===")

if workflow_results:
    # シャープレシオのヒートマップ
    fig1 = workflow_results.plot_heatmap(
        x_param="ma_short",
        y_param="ma_long",
        metric="sharpe_ratio",
    )
    fig1.update_layout(height=400, title="シャープレシオ by MA期間")
    fig1.show()
    
    # 勝率のヒートマップ
    fig2 = workflow_results.plot_heatmap(
        x_param="ma_short",
        y_param="stop_loss",
        metric="win_rate",
    )
    fig2.update_layout(height=400, title="勝率 by MA短期 x 損切り")
    fig2.show()

=== Step4: ヒートマップ可視化 ===


In [31]:
# Step5: 結果保存
print("=== Step5: 結果保存 ===")

if workflow_results:
    output_dir = Path("output")
    output_dir.mkdir(exist_ok=True)
    
    json_path = workflow_results.save(output_dir / "workflow_optimization.json")
    csv_path = workflow_results.save(output_dir / "workflow_optimization.csv")
    
    print(f"JSON保存: {json_path}")
    print(f"CSV保存: {csv_path}")

=== Step5: 結果保存 ===
JSON保存: output/workflow_optimization.json
CSV保存: output/workflow_optimization.csv


---
## 19. エラーハンドリング

最適化で発生する可能性のある例外と対処法です。

| 例外 | 発生条件 | 対処法 |
|------|----------|--------|
| `InvalidSearchSpaceError` | 探索空間が未定義 | `add_search_space()`で定義 |
| `NoValidParametersError` | 制約適用後に有効な組み合わせなし | 制約条件を緩和 |
| `OptimizationTimeoutError` | タイムアウト超過 | timeout延長または探索空間縮小 |

In [32]:
# InvalidSearchSpaceError: 探索空間未定義
try:
    empty_opt = StrategyOptimizer()
    empty_opt.run(
        symbols=["7203"],
        start="2024-01-01",
        end="2024-12-31",
    )
except InvalidSearchSpaceError as e:
    print(f"InvalidSearchSpaceError: {e}")
    print("→ add_search_space()で探索空間を定義してください")

InvalidSearchSpaceError: No search spaces defined
→ add_search_space()で探索空間を定義してください


In [33]:
# NoValidParametersError: 制約で全て除外
try:
    strict_opt = StrategyOptimizer()
    strict_opt.add_search_space("ma_short", [50, 75])
    strict_opt.add_search_space("ma_long", [10, 20])
    strict_opt.add_constraint(lambda p: p["ma_short"] < p["ma_long"])  # 全て除外
    
    strict_opt.run(
        symbols=["7203"],
        start="2024-01-01",
        end="2024-12-31",
    )
except NoValidParametersError as e:
    print(f"NoValidParametersError: {e}")
    print("→ 制約条件を緩和してください")

NoValidParametersError: No valid parameter combinations after applying constraints
→ 制約条件を緩和してください


In [34]:
# OptimizationTimeoutError: タイムアウト
try:
    timeout_opt = StrategyOptimizer()
    timeout_opt.add_search_space("ma_short", [5, 10, 20, 25, 30])
    timeout_opt.add_search_space("ma_long", [50, 75, 100, 150, 200])
    timeout_opt.add_search_space("stop_loss", [-0.05, -0.08, -0.10, -0.12, -0.15])
    timeout_opt.add_constraint(lambda p: p["ma_short"] < p["ma_long"])
    
    timeout_opt.run(
        symbols=["7203", "9984", "6758"],
        start="2024-01-01",
        end="2024-12-31",
        timeout=1.0,  # 非常に短いタイムアウト
    )
except OptimizationTimeoutError as e:
    print(f"OptimizationTimeoutError: {e}")
    print(f"  完了: {e.completed}/{e.total}件")
    print("→ timeoutを延長するか、探索空間を縮小してください")

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

...

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

OptimizationTimeoutError: Optimization timed out after 1.0s: completed 1/125 trials
  完了: 1/125件
→ timeoutを延長するか、探索空間を縮小してください


---
## 20. Tips & ベストプラクティス

### 探索空間のサイズ
- グリッドサーチは組み合わせ数が爆発的に増加
- 100通り以上はランダムサーチを検討
- まず粗い探索 → 絞り込んで細かい探索

### 過学習の防止
- `validation="walk_forward"`を使用
- OOSメトリクスとISメトリクスの乖離をチェック
- 複数銘柄・期間でテスト

### パフォーマンス
- `n_jobs=-1`で全CPUコア使用
- 長時間実行は`streaming_output`でバックアップ
- 大規模探索は`timeout`で制限

In [35]:
# 段階的探索の例
print("=== 段階的探索 ===")

# Phase 1: 粗い探索
print("\nPhase 1: 粗い探索")
opt_phase1 = StrategyOptimizer()
opt_phase1.add_search_space("ma_short", [5, 15, 25])
opt_phase1.add_search_space("ma_long", [50, 100, 200])
opt_phase1.add_constraint(lambda p: p["ma_short"] < p["ma_long"])

try:
    res_phase1 = opt_phase1.run(
        symbols=["7203"],
        start="2024-01-01",
        end="2024-12-31",
    )
    best1 = res_phase1.best()
    if best1:
        print(f"Phase 1 最良: {best1.params}")
        
        # Phase 2: 最良付近を細かく探索
        print("\nPhase 2: 細かい探索")
        ma_short_best = best1.params["ma_short"]
        ma_long_best = best1.params["ma_long"]
        
        opt_phase2 = StrategyOptimizer()
        opt_phase2.add_search_space(
            "ma_short",
            [max(3, ma_short_best - 5), ma_short_best, ma_short_best + 5],
        )
        opt_phase2.add_search_space(
            "ma_long",
            [max(30, ma_long_best - 25), ma_long_best, ma_long_best + 25],
        )
        opt_phase2.add_constraint(lambda p: p["ma_short"] < p["ma_long"])
        
        res_phase2 = opt_phase2.run(
            symbols=["7203"],
            start="2024-01-01",
            end="2024-12-31",
        )
        best2 = res_phase2.best()
        if best2:
            print(f"Phase 2 最良: {best2.params}")
            print(f"シャープレシオ改善: {best1.metrics['sharpe_ratio']:.2f} → {best2.metrics['sharpe_ratio']:.2f}")
except Exception as e:
    print(f"エラー: {e}")

=== 段階的探索 ===

Phase 1: 粗い探索


Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

...

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/244 [00:00<?, ?bar/s]

Phase 2 最良: {'ma_short': 25, 'ma_long': 100}
シャープレシオ改善: 2.15 → 2.15


---
## 21. まとめ

このノートブックでは、`StrategyOptimizer`を使った戦略パラメータ最適化を学びました。

### 機能一覧

| 機能 | 説明 |
|------|------|
| `add_search_space()` | パラメータ候補値を定義 |
| `add_constraint()` | 無効な組み合わせを除外 |
| `run(method="grid")` | 全組み合わせ探索 |
| `run(method="random")` | ランダムサンプリング |
| `run(validation="walk_forward")` | 過学習対策 |
| `run(timeout=...)` | タイムアウト設定 |
| `run(streaming_output=...)` | 逐次保存 |
| `results.best()` | 最良結果取得 |
| `results.top(n)` | 上位N件取得 |
| `results.plot_heatmap()` | 可視化 |
| `results.save()` | JSON/CSV保存 |
| `OptimizationResults.load()` | 読み込み |

### 次のステップ
- 独自の戦略パラメータで最適化を実行
- ウォークフォワードで過学習をチェック
- 複数銘柄での汎化性能を確認
- バックテスト（`backtester_sample.ipynb`）と組み合わせて運用